# 导入tushare

In [ ]:
import tushare as ts
import decimal
from decimal import Decimal

# 初始化pro接口

In [ ]:
pro = ts.pro_api('ec30ba0eb5f07c64bf98162161fd2009bbd3ca48b7c02f3a81b99f97')

# 查询当前所有正常上市交易的股票列表

In [ ]:
stockBasicData = pro.stock_basic(exchange='', list_status='L')

In [ ]:
print(type(stockBasicData.columns.values),stockBasicData.columns.values)

# 获取股票所属市场

In [ ]:
def getMarketOfStock(ts_code):
    for row in stockBasicData.itertuples():
        if getattr(row, 'ts_code') == ts_code:
            return getattr(row, 'market')

# 获取股票基本信息

In [ ]:
def getBasicOfStock(ts_code):
    for row in stockBasicData.itertuples():
        if getattr(row, 'ts_code') == ts_code:
            print(ts_code, getattr(row, 'name'),getattr(row, 'area'), getattr(row, 'industry'))

# 获取涨停板系数

In [ ]:
def getIncrease(market):
    if market == '主板':
        return '1.1'
    elif market == '创业板':
        return '1.2'
    elif market == '科创板':
        return '1.2'
    elif market == '北交所':
        return '1.3'
    else :
        return '0'
    

In [ ]:
decimal.getcontext().rounding = "ROUND_HALF_UP"

def isGoodJobCal(openPrice,closePrice, increase):
    limitUp = (Decimal(str(openPrice))*Decimal(increase)).quantize(decimal.Decimal("0.00"))
    if Decimal(str(closePrice)) == limitUp:
        return True
    else :
        return False

# 拉取数据

In [ ]:
todayData = pro.daily(**{
    "ts_code": "",
    "trade_date": "",
    "start_date": "",
    "end_date": "",
    "offset": "",
    "limit": ""
}, fields=[
    "ts_code",
    "trade_date",
    "open",
    "high",
    "low",
    "close",
    "pre_close",
    "change",
    "pct_chg",
    "vol",
    "amount"
])
print(todayData)

In [ ]:
for row in todayData.itertuples():
    if getattr(row, 'high') == getattr(row, 'close'):
        if isGoodJobCal(getattr(row, 'pre_close'), getattr(row, 'close'),getIncrease(getMarketOfStock(getattr(row, 'ts_code')))):
            print("Good Job:",getattr(row, 'ts_code'),"deal amount:", round(getattr(row, 'amount')/100000, 3),"亿元")
            getBasicOfStock(getattr(row, 'ts_code'))
            print("")